In [1]:
from itertools import product
from sys import stdout as out
from mip import Model, xsum, minimize, BINARY

# Nomes dos lugares a serem visitados
places = ['Antwerp', 'Bruges', 'C-Mine', 'Dinant', 'Ghent',
          'Grand-Place de Bruxelles', 'Hasselt', 'Leuven',
          'Mechelen', 'Mons', 'Montagne de Bueren', 'Namur',
          'Remouchamps', 'Waterloo']

# Distâncias em uma matriz triangular
dists = [[83, 81, 113, 52, 42, 73, 44, 23, 91, 105, 90, 124, 57],
         [161, 160, 39, 89, 151, 110, 90, 99, 177, 143, 193, 100],
         [90, 125, 82, 13, 57, 71, 123, 38, 72, 59, 82],
         [123, 77, 81, 71, 91, 72, 64, 24, 62, 63],
         [51, 114, 72, 54, 69, 139, 105, 155, 62],
         [70, 25, 22, 52, 90, 56, 105, 16],
         [45, 61, 111, 36, 61, 57, 70],
         [23, 71, 67, 48, 85, 29],
         [74, 89, 69, 107, 36],
         [117, 65, 125, 43],
         [54, 22, 84],
         [60, 44],
         [97],
         []]

# Números de nós e listas de vértices
n, V = len(dists), set(range(len(dists)))

# Matriz de distâncias 
c = [[0 if i == j
      else dists[i][j-i-1] if j > i
      else dists[j][i-j-1]
      for j in V] for i in V]

model = Model()

# Variável binária indicando se(i,j) está usando ou não
x = [[model.add_var(var_type=BINARY) for j in V] for i in V]

# variável contínua para evitar ciclos: cada cidade terá um id sequencial diferente na rota planejada, 
# exceto a primeira
y = [model.add_var() for i in V]

# Função objetivo: minimizar a distância
model.objective = minimize(xsum(c[i][j]*x[i][j] for i in V for j in V))

# restrição: sair de cada cidade apenas uma vez
for i in V:
    model += xsum(x[i][j] for j in V - {i}) == 1

# restrição: entrar em cada cidade apenas uma vez
for i in V:
    model += xsum(x[j][i] for j in V - {i}) == 1

# Eliminação do ciclo
for (i, j) in product(V - {0}, V - {0}):
    if i != j:
        model += y[i] - (n+1)*x[i][j] >= y[j]-n

# otimização
model.optimize()

# Verificando se a solução foi achada
if model.num_solutions:
    out.write('Rota com distância total %g achada: %s'
              % (model.objective_value, places[0]))
    nc = 0
    while True:
        nc = [i for i in V if x[nc][i].x >= 0.99][0]
        out.write(' -> %s' % places[nc])
        if nc == 0:
            break
    out.write('\n')

Welcome to the CBC MILP Solver 
Version: devel 
Build Date: Nov 15 2020 

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 184 (0) rows, 195 (-15) columns and 832 (0) elements
Clp1000I sum of infeasibilities 8.22201e-05 - average 4.46848e-07, 124 fixed columns
Coin0506I Presolve 182 (-2) rows, 69 (-126) columns and 474 (-358) elements
Clp0029I End of values pass after 69 iterations
Clp0014I Perturbing problem by 0.001% of 5.362616 - largest nonzero change 3.4125692e-05 ( 0.00073948616%) - largest zero change 1.8240846e-05
Clp0000I Optimal - objective value 399.2
Clp0000I Optimal - objective value 399.2
Coin0511I After Postsolve, objective 399.2, infeasibilities - dual 0 (0), primal 0 (0)
Clp0000I Optimal - objective value 399.2
Clp0000I Optimal - objective value 399.2
Clp0000I Optimal - objective value 399.2
Coin0511I After Postsolve, objective 399.2, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 399.2 - 0 i